# OS11 - Exercises 9-11

In [1]:
from sympy import *

In [2]:
import numpy as np

In [3]:
import scipy.linalg as linalg

In [4]:
from scipy.optimize import linprog

### Previous functions useful here

In [5]:
def nodes_indexer(named_nodes):
    nodes = {}
    for i in range(len(named_nodes)):
        nodes[named_nodes[i]] = i
    return nodes

In [6]:
def ANMatrix_from_arcs(graph, num_nodes):
    num_arcs = len(graph)
    arcs = []
    MN = np.zeros((num_nodes, num_arcs)).astype(int)
    for a in range(len(graph)):
        MN[graph[a][0][0], a] = 1
        MN[graph[a][0][1], a] = -1
        arcs.append(graph[a][0])
    return MN

In [29]:
def print_path(graph, selected_nodes):
    s = ''
    sep = ' -> '
    s += sep
    for i in range(len(graph)):
        if selected_nodes[i] > 0.1:
            s += str(graph[i][0]) + sep
    return s

## Ex. 8 - SP time-restricted 

In [7]:
named_nodes = ['s', '2', '3', '4', '5', 't']   #node enumeration. Can be any non repeated inmutable.

In [8]:
nodes = nodes_indexer(named_nodes)

In [9]:
gen_graph = []

gen_graph.append((('s', '2'), (2,3)))   #distance, time
gen_graph.append((('s', '3'), (1,1)))
gen_graph.append((('2', '4'), (2,3)))
gen_graph.append((('2', 't'), (5,1)))
gen_graph.append((('5', '5'), (2,3)))
gen_graph.append((('5', 't'), (2,5)))

gen_graph


[(('s', '2'), (2, 3)),
 (('s', '3'), (1, 1)),
 (('2', '4'), (2, 3)),
 (('2', 't'), (5, 1)),
 (('5', '5'), (2, 3)),
 (('5', 't'), (2, 5))]

Graph data is in named form, I need it in indexed form:

In [11]:
indexed_graph = [((nodes[a[0][0]], nodes[a[0][1]]), a[1]) for a in gen_graph]
indexed_graph

[((0, 1), (2, 3)),
 ((0, 2), (1, 1)),
 ((1, 3), (2, 3)),
 ((1, 5), (5, 1)),
 ((4, 4), (2, 3)),
 ((4, 5), (2, 5))]

Now, I can create the arcs-nodes matrix

In [13]:
A = ANMatrix_from_arcs(indexed_graph, len(nodes))
A

array([[ 1,  1,  0,  0,  0,  0],
       [-1,  0,  1,  1,  0,  0],
       [ 0, -1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  1],
       [ 0,  0,  0, -1,  0, -1]])

Cost function:

In [17]:
costos = [a[1][0] for a in gen_graph]
c = np.array(costos)

c

array([2, 1, 2, 5, 2, 2])

El verctor de igualdades b:
    

In [15]:
b = np.zeros((len(nodes)))
b[nodes['s']] = 1 
b[nodes['t']] = -1 

b

array([ 1.,  0.,  0.,  0.,  0., -1.])

Límites para cada arco:

In [19]:
limites = [(0, None) for a in gen_graph]
limites

[(0, None), (0, None), (0, None), (0, None), (0, None), (0, None)]

Vector de tiempos por cada arco:

In [21]:
t_j = [[a[1][1] for a in gen_graph]]
t_j = np.array(t_j)
t_j

array([[3, 1, 3, 1, 3, 5]])

El tiempo máximo:

In [24]:
T = np.array([9])
T

array([9])

### Resolution

In [25]:
r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, A_ub= t_j, b_ub=T, method='revised simplex')

In [26]:
r

     con: array([0., 0., 0., 0., 0., 0.])
     fun: 7.0
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([5.])
  status: 0
 success: True
       x: array([ 1., -0., -0.,  1.,  0.,  0.])

### Results

In [30]:
print_path(gen_graph, r.x)

" -> ('s', '2') -> ('2', 't') -> "

### Second case, T=8 hs

In [40]:
T = np.array([6])
T

array([6])

In [41]:
r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, A_ub= t_j, b_ub=T, method='simplex')

In [42]:
r

     con: array([0., 0., 0., 0., 0., 0.])
     fun: 7.0
 message: 'Optimization terminated successfully.'
     nit: 7
   slack: array([2.])
  status: 0
 success: True
       x: array([ 1., -0., -0.,  1.,  0.,  0.])

In [43]:
r.x

array([ 1., -0., -0.,  1.,  0.,  0.])